# Photo Classifier: Hyperparameter tuning

## Imports

In [1]:
from IPython.core.interactiveshell import InteractiveShell
%matplotlib inline
%load_ext autoreload
%autoreload 2
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import logging

In [3]:
from file_io import load_pickle_file
from constants import *
from utils import *
from model import get_pretrained_model_for_transfer_learning
from learners.utils import get_learner
from model_optimizer import ModelOptimizer
from image_transforms import IMAGE_TRANSFORMS

In [4]:
if RANDOM_SEED:
    set_random_seeds(RANDOM_SEED)
logging.basicConfig(level=logging.INFO)

## Load model and data

In [5]:
image_data = load_pickle_file(IMAGE_DATA_PATH)

In [6]:
n_classes = len(image_data.labels)
model = get_pretrained_model_for_transfer_learning(n_classes, IS_MULTILABEL_CLASSIFICATION, DROPOUT, MODEL_ARCHITECTURE)
Learner = get_learner(IS_MULTILABEL_CLASSIFICATION)
learner = Learner(model, use_gpu=USE_GPU)

## Hyperparameter tuning

In [7]:
parameter_options = {
    'learning_rate': [0.002*k for k in range(10)],
    'weight_decay': [0.002*k for k in range(10)],
    'epochs': [5, 10],
    'batch_size': [5, 10, 15],
    'image_transforms_training': [IMAGE_TRANSFORMS['training']],
    'image_transforms_validation': [IMAGE_TRANSFORMS['validation']],
    'early_stop_option': [False],
}

In [8]:
optimizer = ModelOptimizer(learner, image_data, parameter_options)
results = optimizer.run_random_search(20)

INFO:model_optimizer:1/20
INFO:learners.base_learner:
            Epoch: 1/10
            Validation loss: 0.36153358221054077
INFO:learners.base_learner:
            Epoch: 2/10
            Validation loss: 0.2032022625207901
INFO:learners.base_learner:
            Epoch: 3/10
            Validation loss: 0.19263756275177002
INFO:learners.base_learner:
            Epoch: 4/10
            Validation loss: 0.18249092996120453
INFO:learners.base_learner:
            Epoch: 5/10
            Validation loss: 0.22006432712078094
INFO:learners.base_learner:
            Epoch: 6/10
            Validation loss: 0.24217696487903595
INFO:learners.base_learner:
            Epoch: 7/10
            Validation loss: 0.20649349689483643
INFO:learners.base_learner:
            Epoch: 8/10
            Validation loss: 0.22513820230960846
INFO:learners.base_learner:
            Epoch: 9/10
            Validation loss: 0.16787296533584595
INFO:learners.base_learner:
            Epoch: 10/10
            V

INFO:learners.base_learner:
            Epoch: 9/10
            Validation loss: 0.38086000084877014
INFO:learners.base_learner:
            Epoch: 10/10
            Validation loss: 0.606271505355835
INFO:learners.base_learner:Lowest validation loss: epoch: 4; loss: 0.26185381412506104
INFO:model_optimizer:10/20
INFO:learners.base_learner:
            Epoch: 1/10
            Validation loss: 0.6232566833496094
INFO:learners.base_learner:
            Epoch: 2/10
            Validation loss: 0.7267290353775024
INFO:learners.base_learner:
            Epoch: 3/10
            Validation loss: 2.3330416679382324
INFO:learners.base_learner:
            Epoch: 4/10
            Validation loss: 1.3263918161392212
INFO:learners.base_learner:
            Epoch: 5/10
            Validation loss: 0.6725245118141174
INFO:learners.base_learner:
            Epoch: 6/10
            Validation loss: 0.7004094123840332
INFO:learners.base_learner:
            Epoch: 7/10
            Validation loss: 0.59

INFO:learners.base_learner:Lowest validation loss: epoch: 7; loss: 0.2416020929813385
INFO:model_optimizer:19/20
INFO:learners.base_learner:
            Epoch: 1/10
            Validation loss: 0.28696468472480774
INFO:learners.base_learner:
            Epoch: 2/10
            Validation loss: 0.19789057970046997
INFO:learners.base_learner:
            Epoch: 3/10
            Validation loss: 0.1986335664987564
INFO:learners.base_learner:
            Epoch: 4/10
            Validation loss: 0.28258615732192993
INFO:learners.base_learner:
            Epoch: 5/10
            Validation loss: 0.27480262517929077
INFO:learners.base_learner:
            Epoch: 6/10
            Validation loss: 0.23193511366844177
INFO:learners.base_learner:
            Epoch: 7/10
            Validation loss: 0.34938910603523254
INFO:learners.base_learner:
            Epoch: 8/10
            Validation loss: 0.2158111184835434
INFO:learners.base_learner:
            Epoch: 9/10
            Validation loss: 

In [9]:
results

,batch_size,early_stop_option,epochs,image_transforms_training,image_transforms_validation,learning_rate,weight_decay,lowest_validation_loss,time
0,5,False,10,"Compose(\n RandomResizedCrop(size=(256, 256...","Compose(\n Resize(size=256, interpolation=P...",0.002,0.000,0.167873,57.762242
3,5,False,10,"Compose(\n RandomResizedCrop(size=(256, 256...","Compose(\n Resize(size=256, interpolation=P...",0.014,0.018,0.174315,55.714994
1,5,False,10,"Compose(\n RandomResizedCrop(size=(256, 256...","Compose(\n Resize(size=256, interpolation=P...",0.016,0.002,0.179926,54.345328
2,15,False,5,"Compose(\n RandomResizedCrop(size=(256, 256...","Compose(\n Resize(size=256, interpolation=P...",0.006,0.016,0.180700,25.453360
6,5,False,10,"Compose(\n RandomResizedCrop(size=(256, 256...","Compose(\n Resize(size=256, interpolation=P...",0.016,0.002,0.188843,55.235379
18,15,False,10,"Compose(\n RandomResizedCrop(size=(256, 256...","Compose(\n Resize(size=256, interpolation=P...",0.014,0.000,0.197891,49.529618
11,10,False,10,"Compose(\n RandomResizedCrop(size=(256, 256...","Compose(\n Resize(size=256, interpolation=P...",0.006,0.000,0.201503,53.611054
5,15,False,10,"Compose(\n RandomResizedCrop(size=(256, 256...","Compose(\n Resize(size=256, interpolation=P...",0.002,0.010,0.215282,50.769646
7,10,False,5,"Compose(\n RandomResizedCrop(size=(256, 256...","Compose(\n Resize(size=256, interpolation=P...",0.002,0.000,0.226063,26.269106
4,10,False,5,"Compose(\n RandomResizedCrop(size=(256, 256...","Compose(\n Resize(size=256, interpolation=P...",0.008,0.004,0.241528,26.473193


In [10]:
best_learner = optimizer.get_best_learner()